In [17]:
from faster_rcnn import config
from faster_rcnn import vgg
from faster_rcnn import roi_helpers
from faster_rcnn import data_generators

import numpy as np
import pickle
import os
import time
import cv2
from imp import reload

from keras.layers import Input
from keras.models import Model

In [18]:
# 用于修改模块后重新加载
reload(config)
reload(vgg)
reload(roi_helpers)
reload(data_generators)

<module 'faster_rcnn.data_generators' from 'C:\\Users\\paops\\Desktop\\sqdt\\vehicledetector\\faster_rcnn\\data_generators.py'>

In [19]:
# 图像预处理
def format_img(img, C):
    img_min_side = C.im_size
    (height,width,_) = img.shape
    (resized_width, resized_height, ratio) = data_generators.get_new_img_size(width, height, img_min_side)
    img = cv2.resize(img, (resized_width, resized_height), interpolation=cv2.INTER_CUBIC) # 图像resize
    img = data_generators.normalize_img(img, C) # 图像通道归一化
    return img, ratio

In [20]:
# 得到bounding boxes在原图像上的尺寸参数
def get_real_coordinates(ratio, x1, y1, x2, y2):

    real_x1 = round(x1 / ratio)
    real_y1 = round(y1 / ratio)
    real_x2 = round(x2 / ratio)
    real_y2 = round(y2 / ratio)

    return (real_x1, real_y1, real_x2, real_y2)

In [21]:
# 建立模型
def get_models(C):
    # 定义输入维度
    img_input = Input(shape = (None, None, 3))
    roi_input = Input(shape = (C.num_rois, 4))
    feature_map_input = Input(shape = (None, None, 512))

    # 定义共享层
    shared_layers = vgg.nn_base(img_input, trainable = False)

    # 定义共享层基础上的rpn层
    num_anchors = len(C.anchor_box_scales) * len(C.anchor_box_ratios)
    rpn = vgg.rpn(shared_layers, num_anchors)

    # 定义基于特征图的分类层
    classifier = vgg.classifier(feature_map_input, roi_input, C.num_rois, nb_classes = len(C.class_mapping), trainable = False)

    model_rpn = Model(img_input, rpn)
    model_classifier_only = Model([feature_map_input, roi_input], classifier)

    # 依据层名载入预训练参数
    model_rpn.load_weights(C.model_path, by_name = True)
    model_classifier_only.load_weights(C.model_path, by_name = True)

    model_rpn.compile(optimizer = 'sgd', loss = 'mse')
    model_classifier_only.compile(optimizer = 'sgd', loss = 'mse')
    return model_rpn, model_classifier_only

In [22]:
# 实现图像检测与分类
def detect_predict(img, C, model_rpn, model_classifier_only, bbox_threshold, class_mapping, class_to_color):
    X, ratio = format_img(img, C)

    # 得到特征图及RPN输出
    [Y1, Y2, F] = model_rpn.predict(X)
    
    # 过滤掉与最大置信度区域重合度大于重合阈值的区域
    R = roi_helpers.rpn_to_roi(Y1, Y2, C, overlap_thresh = 0.7)

    # 转换坐标形式(x1,y1,x2,y2)为(x,y,w,h)
    R[:, 2] -= R[:, 0]
    R[:, 3] -= R[:, 1]

    bboxes = {}
    probs = {}
    for jk in range(R.shape[0] // C.num_rois + 1):
        # 将R中得到的每num_rois个bounding boxes作为roi
        ROIs = np.expand_dims(R[C.num_rois * jk:C.num_rois * (jk + 1), :], axis = 0)
        if ROIs.shape[1] == 0:
            break
        
        # 当roi数量不足时，使用重复样本替代
        if jk == R.shape[0] // C.num_rois:
            curr_shape = ROIs.shape
            target_shape = (curr_shape[0], C.num_rois, curr_shape[2])
            ROIs_padded = np.zeros(target_shape).astype(ROIs.dtype)
            ROIs_padded[:, :curr_shape[1], :] = ROIs
            ROIs_padded[0, curr_shape[1]:, :] = ROIs[0, 0, :]
            ROIs = ROIs_padded

        # 通过特征图及RoI输入得到预测分类与回归值
        [P_cls, P_regr] = model_classifier_only.predict([F, ROIs])

        for ii in range(P_cls.shape[1]):
            # 忽略最高分类评价小于bbox阈值或分类评价最高为背景的roi
            if np.max(P_cls[0, ii, :]) < bbox_threshold or np.argmax(P_cls[0, ii, :]) == (P_cls.shape[2] - 1):
                continue

            # 记录类别及对应坐标
            cls_name = class_mapping[np.argmax(P_cls[0, ii, :])]
            if cls_name not in bboxes:
                bboxes[cls_name] = []
                probs[cls_name] = []

            (x, y, w, h) = ROIs[0, ii, :]

            cls_num = np.argmax(P_cls[0, ii, :])
            try:
                (tx, ty, tw, th) = P_regr[0, ii, 4 * cls_num:4 * (cls_num + 1)]
                tx /= C.classifier_regr_std[0]
                ty /= C.classifier_regr_std[1]
                tw /= C.classifier_regr_std[2]
                th /= C.classifier_regr_std[3]
                x, y, w, h = roi_helpers.apply_regr(x, y, w, h, tx, ty, tw, th)
            except:
                pass
            bboxes[cls_name].append([C.rpn_stride * x, C.rpn_stride * y, C.rpn_stride * (x + w), C.rpn_stride * (y + h)])
            probs[cls_name].append(np.max(P_cls[0, ii, :]))

    # 记录预测值
    all_dets = []
    for key in bboxes:
        bbox = np.array(bboxes[key])
        # 使用非极大值抑制消除冗余数据
        new_boxes, new_probs = roi_helpers.non_max_suppression_fast(bbox, np.array(probs[key]), overlap_thresh = 0.2)
        # 仅保留每个类型的前n个最大置信度区域
        n = 3
        for i in range(min(new_boxes.shape[0], n)):
            jk = np.argmax(new_probs)
            # 预测值的阈值
            if new_probs[jk] > 0.4:
                (x1, y1, x2, y2) = new_boxes[jk,:]
                # 得到bounding boxes在原图上的尺寸
                (real_x1, real_y1, real_x2, real_y2) = get_real_coordinates(ratio, x1, y1, x2, y2)
                # 依据类型不同绘制不同颜色矩形框
                img = cv2.rectangle(img, (int(real_x1), int(real_y1)), (int(real_x2), int(real_y2)), (int(class_to_color[key][0]), int(class_to_color[key][1]), int(class_to_color[key][2])), 2)
                # 文本设置
                textLabel = '{}: {}%'.format(key, int(100 * new_probs[jk]))
                all_dets.append((key, 100 * new_probs[jk]))
                (retval,baseLine) = cv2.getTextSize(textLabel, cv2.FONT_HERSHEY_COMPLEX, 1, 1)
                # 防止文本超出图像边界
                if real_y1 < 20 and real_y2 < img.shape[0]:
                    textOrg = (int(real_x1), int(real_y2) + 5)
                elif real_y1 < 20 and real_y2 > img.shape[0]:
                    textOrg = (int(real_x1), img.shape[0] - 10)
                else:
                    textOrg = (int(real_x1), int(real_y1) + 5)
                # 添加文本
                img = cv2.rectangle(img, (textOrg[0] - 5, textOrg[1] + baseLine - 5), (textOrg[0] + retval[0] + 5, textOrg[1] - retval[1] - 5), (0, 0, 0), 2)
                img = cv2.rectangle(img, (textOrg[0] - 5,textOrg[1] + baseLine - 5), (textOrg[0] + retval[0] + 5, textOrg[1] - retval[1] - 5), (255, 255, 255), -1)
                img = cv2.putText(img, textLabel, textOrg, cv2.FONT_HERSHEY_DUPLEX, 1, (0, 0, 0), 1)
                
                new_boxes = np.delete(new_boxes, jk, 0)
                new_probs = np.delete(new_probs, jk, 0)
    
    return img, all_dets

In [26]:
# 读入参数
with open('config.pickle', 'rb') as f_in:
    C = pickle.load(f_in)

# 测试时不进行数据增广
C.use_horizontal_flips = False
C.use_vertical_flips = False
C.rot_90 = False

class_mapping = C.class_mapping
if 'bg' not in class_mapping:
    class_mapping['bg'] = len(class_mapping)
class_mapping = {v: k for k, v in class_mapping.items()} # 将键值对调
class_to_color = {class_mapping[v]: np.random.randint(0, 255, 3) for v in class_mapping} # 给不同类随机分配颜色

model_rpn, model_classifier_only = get_models(C)

img_path = 'data_object_image_2/testing/image_2'
bbox_threshold = 0.75

for idx, img_name in enumerate(sorted(os.listdir(img_path))):
    if idx >= 100:
        break
        
    if not img_name.lower().endswith(('.bmp', '.jpeg', '.jpg', '.png', '.tif', '.tiff')):
        continue
    print(img_name)
    start_time = time.time()
    filepath = os.path.join(img_path, img_name)
    # 读取测试图像
    img = cv2.imread(filepath)
    # 目标检测与分类
    img, all_dets = detect_predict(img, C, model_rpn, model_classifier_only, bbox_threshold, class_mapping, class_to_color)
    
    print('Elapsed time = {}'.format(time.time() - start_time))
    
    # 画出含有目标区域及分类标签的图像
    # cv2.imshow('img', img)
    cv2.imwrite('save/' + img_name, img)

000000.png
Elapsed time = 3.7920262813568115
000001.png
Elapsed time = 3.3156213760375977
000002.png
Elapsed time = 3.3753998279571533
000003.png
Elapsed time = 3.515543222427368
000004.png
Elapsed time = 3.697685956954956
000005.png
Elapsed time = 3.685450792312622
000006.png
Elapsed time = 3.6741061210632324
000007.png
Elapsed time = 3.7768640518188477
000008.png
Elapsed time = 3.7329511642456055
000009.png
Elapsed time = 3.8810861110687256
000010.png
Elapsed time = 3.654474973678589
000011.png
Elapsed time = 3.64860200881958
000012.png
Elapsed time = 3.711942434310913
000013.png
Elapsed time = 3.990938901901245
000014.png
Elapsed time = 3.8653616905212402
000015.png
Elapsed time = 3.799473285675049
000016.png
Elapsed time = 3.8039987087249756
000017.png
Elapsed time = 3.7933576107025146
000018.png
Elapsed time = 3.7689290046691895
000019.png
Elapsed time = 3.766923427581787
000020.png
Elapsed time = 3.8041861057281494
000021.png
Elapsed time = 3.7576351165771484
000022.png
Elapsed t